# Translation

One of the driving forces that has enabled human civilization to develop is the ability to communicate with one another. In most human endeavours, communication is key.

![A multilingual robot](./images/translation.jpg)

Artificial Intelligence (AI) can help simplify communication by translating text or speech between languages, helping to remove barriers to communication across countries and cultures.

## Create a Cognitive Services resource

In Azure, you can use the cognitive services to translate between multiple languages.

If you don't already have one, use the following steps to create a **Cognitive Services** resource in your Azure subscription:

> **Note**: If you already have a Cognitive Services resource, just open its **Quick start** page in the Azure portal and copy its key and endpoint to the cell below. Otherwise, follow the steps below to create one.

1. In another browser tab, open the Azure portal at https://portal.azure.com, signing in with your Microsoft account.
2. Click the **&#65291;Create a resource** button, search for *Cognitive Services*, and create a **Cognitive Services** resource with the following settings:
    - **Subscription**: *Your Azure subscription*.
    - **Resource group**: *Select or create a resource group with a unique name*.
    - **Region**: *Choose any available region*:
    - **Name**: *Enter a unique name*.
    - **Pricing tier**: S0
    - **I confirm I have read and understood the notices**: Selected.
3. Wait for deployment to complete. Then go to your cognitive services resource, and on the **Overview** page, click the link to manage the keys for the service. You will need the endpoint and keys to connect to your cognitive services resource from client applications.

### Get the Key and Location for your Cognitive Services resource

To use your cognitive services resource, client applications need its authentication key and location:

1. In the Azure portal, on the **Keys and Endpoint** page for your cognitive service resource, copy the **Key1** for your resource and paste it in the code below, replacing **YOUR_COG_KEY**.
2. Copy the **Location** for your resource and and paste it in the code below, replacing **YOUR_COG_LOCATION**.
>**Note**: Stay on the **Keys and Endpoint** page and copy the **Location** from this page (example: _westus_). Please _do not_ add spaces between words for the Location field. 
3. Run the code below by clicking the **Run cell** (&#9655;) button to the left of the cell.

In [1]:
cog_key = '908e068154524fa0ab567babf0ae9783'
cog_location = 'southeastasia'

print('Ready to use cognitive services in {} using key {}'.format(cog_location, cog_key))

Ready to use cognitive services in southeastasia using key 908e068154524fa0ab567babf0ae9783


## Translating Text

As its name suggests, the **Translator Text** service enables you to translate text from one language to another.

There's no Python SDK for this service, but you can use its REST interface to submit requests to an endpoint over HTTP, which is relatively easy to do in Python by using the **requests** library. The information about the text to be translated and the resulting translated text are exchanged in JSON format.

Run the following cell to create a function that does this, and then test the function with a simple translation from English to French.

In [2]:
# Create a function that makes a REST request to the Text Translation service
def translate_text(cog_location, cog_key, text, to_lang='fr', from_lang='en'):
    import requests, uuid, json

    # Create the URL for the Text Translator service REST request
    path = 'https://api.cognitive.microsofttranslator.com/translate?api-version=3.0'
    params = '&from={}&to={}'.format(from_lang, to_lang)
    constructed_url = path + params

    # Prepare the request headers with Cognitive Services resource key and region
    headers = {
        'Ocp-Apim-Subscription-Key': cog_key,
        'Ocp-Apim-Subscription-Region':cog_location,
        'Content-type': 'application/json',
        'X-ClientTraceId': str(uuid.uuid4())
    }

    # Add the text to be translated to the body
    body = [{
        'text': text
    }]

    # Get the translation
    request = requests.post(constructed_url, headers=headers, json=body)
    response = request.json()
    return response[0]["translations"][0]["text"]


# Test the function
text_to_translate = "Hello"

translation = translate_text(cog_location, cog_key, text_to_translate, to_lang='fr', from_lang='en')
print('{} -> {}'.format(text_to_translate,translation))

Hello -> Bonjour


The service should have translated the English text "Hello" to the French "Bonjour".

Notice that the languages are specified by using a standard system of language abbreviations, with *en* for English and *fr* for French. You can also use abbreviations that include specific cultures, which is useful when the same language is used in different geographical areas - often with different spellings. For example *en-US* denotes English in the Unites States, while *en-GB* indicates English in Great Britain.

Run the following cell to translate between British English and Italian.

In [3]:
text_to_translate = "Hello"

translation = translate_text(cog_location, cog_key, text_to_translate, to_lang='it-IT', from_lang='en-GB')
print('{} -> {}'.format(text_to_translate,translation))

Hello -> Ciao


Let's try another translation - this time from US English to Chinese.

In [4]:
text_to_translate = "Hello"

translation = translate_text(cog_location, cog_key, text_to_translate, to_lang='zh-CN', from_lang='en-US')
print('{} -> {}'.format(text_to_translate,translation))

Hello -> 你好


## Speech Translation

You can use the **Speech** service to translate spoken language.

Now you can run the following cell to create and test a function that uses the Speech SDK to translate audible speech.

In [5]:
# Create a function to translate audio in one language to text in another
def translate_speech(cog_location, cog_key, audio_file=None, to_lang='fr-FR', from_lang='en-US'):
    from azure.cognitiveservices.speech import SpeechConfig, AudioConfig, ResultReason
    from azure.cognitiveservices.speech.translation import SpeechTranslationConfig, TranslationRecognizer

    # Configure the speech translation service
    translation_config = SpeechTranslationConfig(subscription=cog_key, region=cog_location)
    translation_config.speech_recognition_language = from_lang
    translation_config.add_target_language(to_lang)

    # Configure audio input
    if audio_file is None:
        audio_config = AudioConfig() # Use default input (microphone)
    else:
        audio_config = AudioConfig(filename=audio_file) # Use file input

    # Create a translation recognizer and use it to translate speech input
    recognizer = TranslationRecognizer(translation_config, audio_config)
    result = recognizer.recognize_once()

    # Did we get it?
    translation = ''
    speech_text = ''
    if result.reason == ResultReason.TranslatedSpeech:
        speech_text = result.text
        translation =  result.translations[to_lang]
    elif result.reason == ResultReason.RecognizedSpeech:
        speech_text = result.text
        translation =  'Unable to translate speech'
    else:
        translation = 'Unknown'
        speech_text = 'Unknown'

    # rturn the translation
    return speech_text, translation
    

# Test the function
import os

file_name = 'english.wav'
file_path = os.path.join('data', 'translation', file_name)
speech, translated_speech = translate_speech(cog_location, cog_key, file_path, to_lang='es', from_lang='en-US')
result = '{} -> {}'.format(speech, translated_speech)

# Show translated text
print(result)

Hello. -> Hola.


Note that the "to" language must be identified using a 2-character language code (for example *en*), while the "from" language must include the culture indicator (for example *en-US*).

Let's try translating from French to English.

In [6]:
import os

file_name = 'french.wav'
file_path = os.path.join('data', 'translation', file_name)
speech, translated_speech = translate_speech(cog_location, cog_key, file_path, to_lang='en', from_lang='fr-FR')
result = '{} -> {}'.format(speech, translated_speech)

# Show translated text
print(result)

Danger. -> danger.


## Learn More

You can learn more about the [Translator Text](https://docs.microsoft.com/azure/cognitive-services/translator/) and [translation with the Speech service](https://docs.microsoft.com/azure/cognitive-services/speech-service/index-speech-translation) in the service documentation.